In [1]:
import datetime
import logging
import os
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import peilbeheerst_model.ribasim_parametrization as ribasim_param
import ribasim
import ribasim.nodes
from peilbeheerst_model.add_storage_basins import AddStorageBasin
from peilbeheerst_model.controle_output import *
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")


In [2]:
# import datetime
# import logging
# import os
# import pathlib
# import sys
# import warnings
# from pathlib import Path

# import pandas as pd
# import ribasim
# import ribasim.nodes

# #load ribasim_nl
# module_path = Path.cwd() / '../../ribasim_nl/'
# sys.path.append(str(module_path))

# current_dir = os.getcwd()
# parent_dir = os.path.dirname(current_dir)
# if parent_dir not in sys.path:
#     sys.path.append(parent_dir)

# %reload_ext autoreload
# %autoreload 2
# warnings.filterwarnings('ignore')

# import peilbeheerst_model.ribasim_parametrization as ribasim_param
# from peilbeheerst_model.add_storage_basins import AddStorageBasin
# from peilbeheerst_model.controle_output import *
# from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor



## Define variables and model

#### Set Config

In [3]:
# Set paths
# work_dir = pathlib.Path("../../../../../Ribasim_networks/Waterschappen/AmstelGooienVecht/modellen/AmstelGooienVecht_boezemmodel")
waterschap = "AmstelGooienVecht"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = 0  # default LevelBoundary level


## Process the feedback form

In [4]:
name = "Ron Bruijns (HKV)"
versie = "2024_8_8"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed
)
processor.run()


Processed all actions


#### Load model

In [5]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)


## Model specific tweaks

In [6]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

In [7]:
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["node_id"] == 195, "meta_streefpeil"] = -2.45
ribasim_model.basin.state.df.loc[ribasim_model.basin.state.df.node_id == 195, "level"] = -2.45


In [8]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

# Parameterization

## Nodes

### Basin (characteristics)

In [9]:
# Define the initial state of each basin. Is set equal to the streefpeil
# ribasim_param.set_initial_basin_state(ribasim_model) #the initial states of the basins are by default already set to the streefpeil!



In [10]:
# Add storage basins
model_name = "AmstelGooienVecht_StorageBasins"
# node_ids = [1, 2, 3]  # Specify node IDs to process or include_hoofdwater Boolean
# processor = AddStorageBasin(ribasim_toml, model_name, output_folder, include_hoofdwater=True, log=True, node_ids=None)
# ribasim_model = processor.run()

processor = AddStorageBasin(
    path_ribasim_toml, model_name, output_folder, include_hoofdwater=False, log=False, node_ids=None
)
ribasim_model = processor.run()

# Check basin area
ribasim_param.validate_basin_area(ribasim_model)


INFO:root:Processing Basin Node ID: 1
INFO:root:Successfully added Manning Resistance node with Node ID: 1038
INFO:root:Successfully added new basin node with Node ID: 1039
INFO:root:Connected new Basin Node ID: 1039 to original Basin Node ID: 1 via Manning Resistance Node ID: 1038
INFO:root:Processing Basin Node ID: 2
INFO:root:Successfully added Manning Resistance node with Node ID: 1040
INFO:root:Successfully added new basin node with Node ID: 1041
INFO:root:Connected new Basin Node ID: 1041 to original Basin Node ID: 2 via Manning Resistance Node ID: 1040
INFO:root:Processing Basin Node ID: 3
INFO:root:Successfully added Manning Resistance node with Node ID: 1042
INFO:root:Successfully added new basin node with Node ID: 1043
INFO:root:Connected new Basin Node ID: 1043 to original Basin Node ID: 3 via Manning Resistance Node ID: 1042
INFO:root:Processing Basin Node ID: 4
INFO:root:Successfully added Manning Resistance node with Node ID: 1044
INFO:root:Successfully added new basin no

All basins are larger than 100 m²


In [11]:
# remove the basins of above in the feedback form

In [12]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = unknown_streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"
] = unknown_streefpeil


In [12]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)


### Basin (forcing)

In [13]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(10),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(0),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)


### Pumps

In [14]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut


### Convert all boundary nodes to LevelBoundaries

In [15]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)


### Add Outlet

In [16]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

## Add control, based on the meta_categorie

In [17]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [18]:
ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name


Sturing has been added for the category Inlaat boezem, stuw
Sturing has been added for the category Uitlaat boezem, stuw
Sturing has been added for the category Reguliere stuw


DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name
DEBUG:pyproj:PROJ_ERROR: proj_create: unrecognized format / unknown name


Sturing has been added for the category Inlaat buitenwater peilgebied, stuw
Sturing has been added for the category Uitlaat buitenwater peilgebied, stuw
No stuwen are found in the category of Boezem boezem, stuw
Sturing has been added for the category Inlaat boezem, gemaal
Sturing has been added for the category Uitlaat boezem, gemaal
Sturing has been added for the category Regulier afvoer gemaal
Sturing has been added for the category Regulier aanvoer gemaal
Sturing has been added for the category Uitlaat buitenwater peilgebied, afvoer gemaal
Sturing has been added for the category Uitlaat buitenwater peilgebied, aanvoer gemaal
Sturing has been added for the category Inlaat buitenwater peilgebied, afvoer gemaal
Sturing has been added for the category Inlaat buitenwater peilgebied, aanvoer gemaal
No gemalen are found in the category of Boezem boezem, afvoer gemaal
No gemalen are found in the category of Boezem boezem, aanvoer gemaal


In [19]:
ribasim_model.discrete_control.node.df = ribasim_model.discrete_control.node.df.drop_duplicates(
    subset="node_id"
).reset_index(drop=True)


### Manning Resistance

In [20]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")


# Set numerical settings

In [21]:
# Write model output
ribasim_param.index_reset(ribasim_model)
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2024, 1, 30)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)


INFO:pyogrio._io:Created 2,343 records
INFO:pyogrio._io:Created 2,088 records
INFO:pyogrio._io:Created 443 records


## Run Model

## Iterate over tabulated rating curves

In [22]:
try:
    ribasim_param.iterate_TRC(
        ribasim_param=ribasim_param,
        allowed_tolerance=0.02,
        max_iter=1,
        expected_difference=0.1,
        max_adjustment=0.25,
        cmd=["ribasim", path_ribasim_toml],
        output_dir=output_dir,
        path_ribasim_toml=path_ribasim_toml,
    )

except Exception:
    logging.error("The model was not able to run. Log file:")
    log_file_path = os.path.join(output_dir, "ribasim.log")  # Update with the correct path to your log file
    try:
        with open(log_file_path) as log_file:
            log_content = log_file.read()
            print(log_content)
    except Exception as log_exception:
        logging.error(f"Could not read the log file: {log_exception}")


Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

# Write model

In [23]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap="AmstelGooienVecht",
    modeltype="boezemmodel",
    include_results=True,
)


INFO:pyogrio._io:Created 2,343 records
INFO:pyogrio._io:Created 2,088 records


../../../../../Ribasim_networks/Waterschappen/AmstelGooienVecht/modellen/AmstelGooienVecht_parametrized/results


INFO:pyogrio._io:Created 443 records
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): deltares.thegood.cloud:443
DEBUG:urllib3.connectionpool:https://deltares.thegood.cloud:443 "GET /remote.php/dav/files/nhi_api/Ribasim%20modeldata HTTP/1.1" 200 None
INFO:ribasim_nl.cloud:valid credentials
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): deltares.thegood.cloud:443
DEBUG:urllib3.connectionpool:https://deltares.thegood.cloud:443 "PROPFIND /remote.php/dav/files/nhi_api/Ribasim%20modeldata/AmstelGooienVecht/modellen HTTP/1.1" 207 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): deltares.thegood.cloud:443
DEBUG:urllib3.connectionpool:https://deltares.thegood.cloud:443 "MKCOL /remote.php/dav/files/nhi_api/Ribasim%20modeldata/AmstelGooienVecht/modellen/AmstelGooienVecht_parametrized_2024_8_24 HTTP/1.1" 201 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): deltares.thegood.cloud:443
DEBUG:urllib3.connectionpool:https://deltares.

The model of waterboard AmstelGooienVecht has been uploaded to the goodcloud in the directory of boezemmodel!


In [24]:
stop

NameError: name 'stop' is not defined

## Open Output

In [ ]:
df_basin = pd.read_feather(os.path.join(output_dir, "basin.arrow"))
df_basin


### FlowBoundary

In [ ]:
# Set FlowBoundary rate for each pump
# ribasim_model.flow_boundary.static.df['flow_rate'] = 0 #



### Add Discrete Control

In [ ]:
# Add discrete control nodes and control edges
# ribasim_param.add_discrete_control_nodes(ribasim_model)



### TabulatedRatingCurve

In [ ]:
# # First, set the Q(h)-relationship to a relationship which is at each TRC node the same
# ribasim_param.set_tabulated_rating_curves(ribasim_model, level_increase=1.0, flow_rate=1)



In [ ]:
# # The water level in the boezem is regulated heavily. There is a smaller tolerance for an increase in water level.
# # Hence, the TRC's which discharge water from the boezem to a Terminal should have a higher capacity.
# ribasim_param.set_tabulated_rating_curves_boundaries(ribasim_model, level_increase=0.10, flow_rate=40)


